# Setup

In [ ]:
%matplotlib ipympl
import matplotlib.pyplot

matplotlib.pyplot.ioff()

from package.apps import Apps

# Queries

## Apps using IValidator

In [ ]:
with Apps.init() as apps:
    apps.select(
        {
            "IValidators": lambda app: app.program_cs.custom_app_services(interface_name="IValidator").map(
                lambda class_name: app.cs.flat_map(
                    lambda cs: cs.class_declarations(name=class_name, implements="IValidator")
                ).first
            )
        }
    ).where(lambda app: app["IValidators"].length > 0).table()

## Apps in prod using IFormDataValidator

In [ ]:
with Apps.init() as apps:
    apps.where(lambda app: app.env == "prod").select(
        {"IFormDataValidators": lambda app: app.program_cs.custom_app_services(interface_name="IFormDataValidator")}
    ).where(lambda app: app["IFormDataValidators"].length > 0).table()

## Apps using IPdfFormatter

In [ ]:
with Apps.init() as apps:
    apps.where(lambda app: app.env == "prod").select(
        {
            "Frontend version": lambda app: app.frontend_version,
            "IPdfFormatters": lambda app: app.program_cs.custom_app_services(interface_name="IPdfFormatter").map(
                lambda class_name: app.cs.flat_map(
                    lambda cs: cs.class_declarations(name=class_name, implements="IPdfFormatter")
                ).first
            ),
        }
    ).where(lambda app: app["IPdfFormatters"].length > 0).order_by(
        lambda app: (app["Frontend version"], app.org), reverse=True
    ).table()